Download the [dataset from kaggle](https://www.kaggle.com/c/ncaam-march-mania-2021/data).

In [1]:
import numpy as np
import pandas as pd

In [2]:
fdr = 'ncaam-march-mania-2021/'
def load(fname):
    return pd.read_csv(f'{fdr}{fname}')
seeds = load('MNCAATourneySeeds.csv')
seeds

,Season,Seed,TeamID
0,1985,W01,1207
1,1985,W02,1210
2,1985,W03,1228
3,1985,W04,1260
4,1985,W05,1374
...,...,...,...
2281,2019,Z12,1332
2282,2019,Z13,1414
2283,2019,Z14,1330
2284,2019,Z15,1159


In [3]:
tourney = load('MNCAATourneyCompactResults.csv')
tourney

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,136,1116,63,1234,54,N,0
1,1985,136,1120,59,1345,58,N,0
2,1985,136,1207,68,1250,43,N,0
3,1985,136,1229,58,1425,55,N,0
4,1985,136,1242,49,1325,38,N,0
...,...,...,...,...,...,...,...,...
2246,2019,146,1120,77,1246,71,N,1
2247,2019,146,1277,68,1181,67,N,0
2248,2019,152,1403,61,1277,51,N,0
2249,2019,152,1438,63,1120,62,N,0


In [4]:
regular_season = load('MRegularSeasonDetailedResults.csv')
regular_season

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92827,2020,128,1389,56,1233,54,N,0,15,60,...,16,18,20,4,27,6,17,8,4,20
92828,2020,128,1393,81,1314,53,N,0,28,61,...,16,11,13,13,29,9,18,0,1,18
92829,2020,128,1402,85,1111,68,H,0,34,57,...,15,25,35,8,17,6,18,7,2,18
92830,2020,128,1412,74,1427,69,N,0,26,53,...,27,7,7,12,22,11,13,4,2,18


In [5]:
regular_season.columns

Index(['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc',
       'NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR',
       'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3',
       'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF'],
      dtype='object')

Let's compile season, seeds, winning percentage, points allowed per game, points per game, FG%, FG3%, offensive rebounds per game, defensive rebounds per game, FT%, assists per game, blocks per game, steals per game, fouls per game, and turnovers per game for each team.

In [10]:
class SeasonStats:
    def __init__(self, year):
        self.year = year
        self.data = regular_season[regular_season.Season == year]
        self.stats = pd.DataFrame(columns=['season', 'seed', 'wp', 'papg', 'ppg', 'fgp', 'fgp3', 'orpg', \
                                          'drpg', 'ftp', 'apg', 'bpg', \
                                          'spg', 'fpg', 'topg'])
        self.relevant_teams = pd.unique(tourney[tourney.Season == year+1] \
                                       [['WTeamID', 'LTeamID']].stack())
        
    def get_stats(self):
        for team in self.relevant_teams:
            losing_data = self.data[self.data.LTeamID == team]
            winning_data = self.data[self.data.WTeamID == team]
            n = len(losing_data)+len(winning_data)
            def pg_stat(abbr):
                return (losing_data[f'L{abbr}'].sum() \
                        +winning_data[f'W{abbr}'].sum())/n
            season = self.year
            seed = int(''.join(char for char in seeds[(seeds.Season == self.year+1)&(seeds.TeamID == team)]['Seed'] \
                            .item() if char.isdigit()))
            wp = len(winning_data)/n
            papg = (losing_data['WScore'].sum()+winning_data['LScore'].sum())/n
            ppg = pg_stat('Score')
            fgp = (losing_data['LFGM'].sum()+winning_data['WFGM'].sum())/ \
                    (losing_data['LFGA'].sum()+winning_data['WFGA'].sum())
            fgp3 = (losing_data['LFGM3'].sum()+winning_data['WFGM3'].sum())/ \
                    (losing_data['LFGA3'].sum()+winning_data['WFGA3'].sum())
            orpg = pg_stat('OR')
            drpg = pg_stat('DR')
            ftp = (losing_data['LFTM'].sum()+winning_data['WFTM'].sum())/ \
                    (losing_data['LFTA'].sum()+winning_data['WFTA'].sum())
            apg = pg_stat('Ast')
            bpg = pg_stat('Blk')
            spg = pg_stat('Stl')
            fpg = pg_stat('PF')
            topg = pg_stat('TO')
            to_add = pd.DataFrame([[season, seed, wp, papg, ppg, fgp, fgp3, orpg, drpg, ftp, apg, \
                                    bpg, spg, fpg, topg]], \
                                      columns=self.stats.columns)
            self.stats = self.stats.append(to_add, ignore_index=True)
        self.stats.index = self.relevant_teams
        return self.stats

all_stats = pd.DataFrame(columns=['season', 'seed', 'wp', 'papg', 'ppg', 'fgp', 'fgp3', 'orpg', \
                                          'drpg', 'ftp', 'apg', 'bpg', \
                                          'spg', 'fpg', 'topg'])
for yr in range(2003, 2020):
    all_stats = all_stats.append(SeasonStats(yr).get_stats())
all_stats.to_csv('all_stats.csv')
all_stats
# We need to wait until Sunday on March 14 to know the relevant teams for 2020
# SeasonStats(2020).save_stats()

,season,seed,wp,papg,ppg,fgp,fgp3,orpg,drpg,ftp,apg,bpg,spg,fpg,topg
1197,2003,16,0.586207,71.758621,71.482759,0.430740,0.388350,13.448276,24.448276,0.676724,11.241379,2.103448,7.241379,19.896552,17.482759
1250,2003,16,0.538462,63.807692,63.846154,0.428466,0.350384,10.192308,24.076923,0.729293,12.000000,3.038462,7.076923,18.846154,14.076923
1104,2003,8,0.607143,65.000000,69.285714,0.420362,0.320144,13.571429,23.928571,0.709898,12.107143,3.785714,6.607143,18.035714,13.285714
1356,2003,9,0.800000,68.100000,74.533333,0.472926,0.385263,10.833333,22.466667,0.706186,13.566667,3.200000,7.966667,19.333333,12.500000
1163,2003,2,0.700000,71.400000,80.033333,0.474812,0.386412,14.766667,27.900000,0.674208,15.633333,7.733333,5.933333,18.400000,15.800000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1205,2018,16,0.379310,72.931034,67.034483,0.430666,0.340206,8.758621,25.655172,0.733624,11.896552,2.655172,5.241379,18.448276,13.931034
1439,2018,4,0.656250,71.750000,79.718750,0.497515,0.385144,7.375000,25.625000,0.709531,16.500000,2.281250,5.750000,15.687500,11.937500
1387,2018,13,0.500000,65.843750,65.500000,0.417751,0.320215,11.875000,24.093750,0.652455,11.718750,4.406250,5.906250,18.343750,12.656250
1449,2018,9,0.625000,72.562500,74.281250,0.455087,0.341346,9.781250,23.312500,0.703650,11.593750,5.062500,7.968750,18.750000,13.375000


Let's also add round, winning seed, and losing seed to the tourney results and remove everything else.

In [11]:
tourney['WSeed'] = [seeds[(seeds.TeamID == id)&(seeds.Season == season)]['Seed'].item() \
 for (id, season) in tourney[['WTeamID', 'Season']].values]
tourney['LSeed'] = [seeds[(seeds.TeamID == id)&(seeds.Season == season)]['Seed'].item() \
 for (id, season) in tourney[['LTeamID', 'Season']].values]
def tourney_round(day_num):
    if day_num == 134 or day_num == 135:
        return 0 # play-in
    elif day_num == 136 or day_num == 137:
        return 1 # round of 64
    elif day_num == 138 or day_num == 139:
        return 2 # round of 32
    elif day_num == 143 or day_num == 144:
        return 3 # Sweet Sixteen
    elif day_num == 145 or day_num == 146:
        return 4 # Elite Eight
    elif day_num == 152:
        return 5 # Final Four
    else:
        return 6 # Championship
tourney['Round'] = [tourney_round(day_num) for day_num in tourney['DayNum']]
tourney.to_csv('TourneyCompactAugmented.csv')
tourney

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WSeed,LSeed,Round
0,1985,136,1116,63,1234,54,N,0,X09,X08,1
1,1985,136,1120,59,1345,58,N,0,Z11,Z06,1
2,1985,136,1207,68,1250,43,N,0,W01,W16,1
3,1985,136,1229,58,1425,55,N,0,Y09,Y08,1
4,1985,136,1242,49,1325,38,N,0,Z03,Z14,1
...,...,...,...,...,...,...,...,...,...,...,...
2246,2019,146,1120,77,1246,71,N,1,Y05,Y02,4
2247,2019,146,1277,68,1181,67,N,0,W02,W01,4
2248,2019,152,1403,61,1277,51,N,0,X03,W02,5
2249,2019,152,1438,63,1120,62,N,0,Z01,Y05,5
